In [1]:
# imports
%pylab inline
import numpy as np
from numpy import *
#sys.path.insert(0, '/home/peter/code/projects/tradesys/')
from tradesys import DataMatrix, DateTime, get_stree_log
import tradesys.ml as ml
import talib as ta
from sklearn.preprocessing import scale
import os
import pickle as pkl
from sklearn.preprocessing import scale, robust_scale, normalize
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from statsmodels.tsa.statespace.kalman_filter import KalmanFilter
import collections
import itertools as it
from tqdm import tqdm_notebook as tqdm
import random as rnd
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.svm import LinearSVC
sys.path.append('/home/peter/code/projects')
sys.path.append('/home/ubuntu')
from aidevutil.denoise import remove_noise
from pomegranate import NormalDistribution
from sklearn.linear_model import RidgeClassifier

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.


vizdoom could not be imported


In [2]:
df = pd.read_csv('data/EURUSD_1H.csv')
nd = []
for i,(d,t,o,h,l,c),*rest in df[['Date', 'Time', 'Open', 'High', 'Low', 'Close']].iterrows():
    ds = d.split('-')
    hs = t.split(':')[:]
    ds = [int(x) for x in ds]
    hs = [int(x) for x in hs]
    nd.append(ds + hs + [0] + [o,h,l,c,1])
datamatrix = DataMatrix()
for y,mn,d,hr,m,s, o,h,l,c,v in nd:
    datamatrix.add_row(DateTime(y,mn,d,hr,m,s), o,h,l,c,int(v))

In [3]:
dataname = 'eurusd_1h'

In [4]:
len(datamatrix)

51823

In [5]:
d_open = np.array(datamatrix.open)
d_high = np.array(datamatrix.high)
d_low = np.array(datamatrix.low)
d_close = np.array(datamatrix.close)
d_volume = np.array(datamatrix.volume)

In [6]:
register = {}

In [7]:
periods_to_test = [3, 7, 14, 21, 32, 45, 60, 80]

In [8]:
def regall_single(key, dct, ars):
    for i,a in enumerate(ars):
        d = {**dct}
        d['args'] = (a,)
        register[key+'_'+str(i)] = d    

In [9]:
def signal_maker(s):
    return np.array(get_stree_log(s, datamatrix), dtype=np.float32)

In [10]:
def denoised(x, wlen=128, st=0.8, pd=0.8):
    w = NormalDistribution.from_samples(np.diff(x))
    noise_sample = array([w.sample() for _ in range(10000)])
    o= remove_noise(audio_clip=np.diff(x), noise_clip=noise_sample, n_std_thresh=st, prop_decrease=pd, 
                     n_fft=wlen, win_length=wlen, hop_length=1,verbose=False,visual=False)
    o = hstack([array([0]), o])
    return o

In [11]:
d = { 'maker' : ta.MA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('MA', d, (d_open, d_high, d_low, d_close))

In [12]:
d = { 'maker' : ta.EMA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('EMA', d, (d_open, d_high, d_low, d_close))

In [13]:
d = { 'maker' : ta.DEMA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('DEMA', d, (d_open, d_high, d_low, d_close))

In [14]:
d = { 'maker' : ta.KAMA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('KAMA', d, (d_open, d_high, d_low, d_close))

In [15]:
d = { 'maker' : ta.TEMA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('TEMA', d, (d_open, d_high, d_low, d_close))

In [16]:
d = { 'maker' : ta.TRIMA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('TRIMA', d, (d_open, d_high, d_low, d_close))

In [17]:
d = { 'maker' : ta.WMA,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('WMA', d, (d_open, d_high, d_low, d_close))

In [18]:
d = { 'maker' : ta.T3,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7,
                   'vfactor':.07,
                  }, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                  'vfactor':[0.01, 0.03, 0.05, 0.07, 0.1, 0.15],
                  },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('T3', d, (d_open, d_high, d_low, d_close))

In [19]:
d = { 'maker' : ta.BBANDS,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7,
                   'nbdevup': 1,
                   'nbdevdn': 1,
                   'matype': 0,
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                  'nbdevup': [1, 2, 3],
                  'nbdevdn': [1, 2, 3],
                  'matype': [0],
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    {'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    {'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     },
                    ], 
      }

regall_single('BBANDS', d, (d_open, d_high, d_low, d_close))

In [20]:
d = { 'maker' : ta.MIDPOINT,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('MIDPOINT', d, (d_open, d_high, d_low, d_close))

In [21]:
register['MIDPRICE'] = { 'maker' : ta.MIDPRICE,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

In [22]:
register['SAR'] = { 'maker' : ta.SAR,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'acceleration': .05,
                   'maximum': .5}, # parameters with defaults
       # parameter space for exploration
       'space' : {'acceleration': [0.01, 0.02, 0.05, 0.075, 0.1, 0.2],
                   'maximum': [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

In [23]:
register['SAREXT'] = { 'maker' : ta.SAREXT,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'startvalue':0,
                   'offsetonreverse':0, 
                   'accelerationinitlong':.05,
                   'accelerationlong':.05,
                   'accelerationmaxlong':.5, 
                   'accelerationinitshort':.05, 
                   'accelerationshort':.05,
                   'accelerationmaxshort':.5, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : {'startvalue':[0],
                   'offsetonreverse':[0], 
                   'accelerationinitlong':[0.02, 0.05, 0.075],
                   'accelerationlong':[0.02, 0.05, 0.075,],
                   'accelerationmaxlong':[0.25, 0.5, 0.75], 
                   'accelerationinitshort':[0.02, 0.05, 0.075], 
                   'accelerationshort':[0.02, 0.05, 0.075],
                   'accelerationmaxshort':[0.25, 0.5, 0.75],
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [-1.25, 1.25], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 1,
                     'scale': 0,
                     }], 
      }

In [24]:
register['ADX'] = { 'maker' : ta.ADX,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 14,
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     }], 
      }

In [25]:
register['ADXR'] = { 'maker' : ta.ADXR,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 14,
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     }], 
      }

In [26]:
d = { 'maker' : ta.APO,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'fastperiod': 7, 
                   'slowperiod': 14, 
                   'matype' : 0
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'fastperiod': periods_to_test, 
                   'slowperiod': periods_to_test, 
                   'matype' : [0]
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('APO', d, (d_open, d_high, d_low, d_close))

In [27]:
register['AROON'] = { 'maker' : ta.AROON,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    
                    {'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    ], 
      }

In [28]:
register['AROONOSC'] = { 'maker' : ta.AROONOSC,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [-100, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    ], 
      }

In [29]:
register['CCI'] = { 'maker' : ta.CCI,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    ], 
       }

In [30]:
d = { 'maker' : ta.CMO,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'timeperiod': periods_to_test, 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [-100, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     }], 
      }

regall_single('CMO', d, (d_open, d_high, d_low, d_close))

In [31]:
register['DX'] = { 'maker' : ta.DX,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    ], 
       }

In [32]:
d = {  'maker' : ta.MACD,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'fastperiod': 7, 
                   'slowperiod': 14, 
                   'signalperiod' : 10,
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'fastperiod': periods_to_test, 
                   'slowperiod': periods_to_test, 
                   'signalperiod' : periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }, 
                    
                    {'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    {'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }, 
                    ], 
      }

regall_single('MACD', d, (d_open, d_high, d_low, d_close))

In [33]:
d = {  'maker' : ta.MACDFIX,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {
                   'signalperiod' : 10,
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : {  
                   'signalperiod' : periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }, 
                    
                    {'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    {'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }, 
                    ], 
      }

regall_single('MACDFIX', d, (d_open, d_high, d_low, d_close))

In [34]:
register['MINUS_DI'] = { 'maker' : ta.MINUS_DI,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    ], 
       }

In [35]:
register['PLUS_DI'] = { 'maker' : ta.PLUS_DI,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    ], 
       }

In [36]:
register['MINUS_DM'] = { 'maker' : ta.MINUS_DM,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    ], 
       }

In [37]:
register['PLUS_DM'] = { 'maker' : ta.PLUS_DM,  # function to make the arrays
       'args' : (d_high, d_low), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    ], 
       }

In [38]:
d = { 'maker' : ta.MOM,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('MOM', d, (d_open, d_high, d_low, d_close))

In [39]:
d = { 'maker' : ta.PPO,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'fastperiod': 7, 
                   'slowperiod': 14, 
                   'matype' : 0}, # parameters with defaults
       # parameter space for exploration
       'space' : {'fastperiod': periods_to_test, 
                   'slowperiod': periods_to_test, 
                   'matype' : [0]},
       'outputs' : [{'range': [-1, 1], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 1,
                     'scale': 1,                     
                     }], # how many output arrays it returns and their output range, if any
      }

regall_single('PPO', d, (d_open, d_high, d_low, d_close))

In [40]:
d = { 'maker' : ta.ROC,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('ROC', d, (d_open, d_high, d_low, d_close))

In [41]:
d = { 'maker' : ta.ROCP,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('ROCP', d, (d_open, d_high, d_low, d_close))

In [42]:
d = { 'maker' : ta.ROCR,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('ROCR', d, (d_open, d_high, d_low, d_close))

In [43]:
d = { 'maker' : ta.ROCR100,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('ROCR100', d, (d_open, d_high, d_low, d_close))

In [44]:
d = { 'maker' : ta.RSI,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     }], 
      }

regall_single('RSI', d, (d_open, d_high, d_low, d_close))

In [45]:
register['STOCH'] = { 'maker' : ta.STOCH,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'fastk_period': 10, 
                   'slowk_period': 5, 
                   'slowd_period': 5, 
                   'slowd_matype': 0,
                   'slowk_matype': 0, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'fastk_period': periods_to_test, 
                   'slowk_period': periods_to_test, 
                   'slowd_period': periods_to_test, 
                   'slowd_matype': [0],
                   'slowk_matype': [0],
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    {'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    ], 
       }


In [46]:
register['STOCHF'] = { 'maker' : ta.STOCHF,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'fastk_period': 10, 
                   'fastd_period': 5, 
                   'fastd_matype': 0, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'fastk_period': periods_to_test, 
                   'fastd_period': periods_to_test, 
                   'fastd_matype': [0], 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    {'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    ], 
       }

In [47]:
d = { 'maker' : ta.TRIX,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('TRIX', d, (d_open, d_high, d_low, d_close))

In [48]:
d = { 'maker' : ta.TSF,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('TSF', d, (d_open, d_high, d_low, d_close))

In [49]:
register['ULTOSC'] = { 'maker' : ta.ULTOSC,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod1': 10, 
                   'timeperiod2': 20, 
                   'timeperiod3': 30, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'timeperiod1': periods_to_test, 
                   'timeperiod2': periods_to_test, 
                   'timeperiod3': periods_to_test, 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [0, 100], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    
                    ], 
       }

In [50]:
register['WILLR'] = { 'maker' : ta.WILLR,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 10, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'timeperiod': periods_to_test, 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': [-100, 0], 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 0,
                     },
                    
                    ], 
       }

In [51]:
register['ATR'] = { 'maker' : ta.ATR,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 10, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'timeperiod': periods_to_test, 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    ], 
       }


In [52]:
register['NATR'] = { 'maker' : ta.NATR,  # function to make the arrays
       'args' : (d_high, d_low, d_close), # arguments for the function
       'kwargs' : {'timeperiod': 10, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'timeperiod': periods_to_test, 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    ], 
       }

In [53]:
register['BETA'] = { 'maker' : ta.BETA,  # function to make the arrays
       'args' : (d_high, d_low,), # arguments for the function
       'kwargs' : {'timeperiod': 10, 
                   }, # parameters with defaults
       # parameter space for exploration
       'space' : { 'timeperiod': periods_to_test, 
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                    
                    ], 
       }

In [54]:
d = { 'maker' : ta.LINEARREG_ANGLE,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('LINEARREG_ANGLE', d, (d_open, d_high, d_low, d_close))

In [55]:
d = { 'maker' : ta.LINEARREG,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('LINEARREG', d, (d_open, d_high, d_low, d_close))

In [56]:
d = { 'maker' : ta.LINEARREG_SLOPE,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('LINEARREG_SLOPE', d, (d_open, d_high, d_low, d_close))

In [57]:
d = { 'maker' : ta.STDDEV,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('STDDEV', d, (d_open, d_high, d_low, d_close))

In [58]:
d = { 'maker' : ta.VAR,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('VAR', d, (d_open, d_high, d_low, d_close))

In [59]:
d = { 'maker' : ta.MAX,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('MAX', d, (d_open, d_high, d_low, d_close))

In [60]:
d = { 'maker' : ta.MIN,  # function to make the arrays
       'args' : (d_open,), # arguments for the function
       'kwargs' : {'timeperiod': 7}, # parameters with defaults
       # parameter space for exploration
       'space' : {'timeperiod': periods_to_test,
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'price',
                     'lag': 1,
                     'abs': 0,
                     'scale': 1,
                     }], 
      }

regall_single('MIN', d, (d_open, d_high, d_low, d_close))

In [61]:
# TradeSys signals

In [62]:
s = """((Price('high') - Price('close')) /
          (Price('high') - Price('low') + F(0.000000001)) * F(100.0))
        """

register['intrabar_percent'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [63]:
s = """ (PRICE('high') - PRICE('low')) / IFTHENELSE( ABS(PRICE('open') - (PRICE('close'))) != F(0.0),
        ABS(PRICE('open') - (PRICE('close'))),
        F(1.0) ) """

register['some_percent'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [64]:
s = """(

        ((PRICE('close', bars_back=1) > PRICE('open', bars_back=1)) and (PRICE('close') < PRICE('open')))

        or

        ((PRICE('close', bars_back=1) < PRICE('open', bars_back=1)) and (PRICE('close') > PRICE('open')))

        )"""

register['prev_opposite_bar'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [65]:
s = """(

        ((PRICE('close', bars_back=1) > PRICE('open', bars_back=1)) and (PRICE('close') > PRICE('open')))

        or

        ((PRICE('close', bars_back=1) < PRICE('open', bars_back=1)) and (PRICE('close') < PRICE('open')))

        )"""

register['prev_same_bar'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [66]:
s = """(((PRICE('close', data={0}) -
                       VALUE_AT_TIME(PRICE('open', data={0}), ['open_time',09:31],
                       days_ago=['days_ago', 0], reset_overnight=true)) /
                       VALUE_AT_TIME(PRICE('open', data={0}), ['open_time',09:31],
                       days_ago=['days_ago', 0], reset_overnight=true))*F(100.0))""".format(0)

register['pmove'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [67]:
s = """(((PRICE('close', data={0}) -
                       VALUE_AT_TIME(PRICE('open', data={0}), ['open_time',04:00],
                       days_ago=['days_ago', 0], reset_overnight=true)) /
                       VALUE_AT_TIME(PRICE('open', data={0}), ['open_time',04:00],
                       days_ago=['days_ago', 0], reset_overnight=true))*F(100.0))""".format(0)

register['pmove2'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [68]:
s = "TIMES_IN_ROW(PRICE('open') < PRICE('close'))"

register['up_bars_in_row'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [69]:
s = "TIMES_IN_ROW(PRICE('open') > PRICE('close'))"

register['down_bars_in_row'] = { 'maker' : signal_maker,  # function to make the arrays 
       'args' : (s,), # arguments for the function
       'kwargs' : {}, # parameters with defaults
       # parameter space for exploration
       'space' : {},
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

In [70]:
# some denoised data
d = { 'maker' : denoised,  # function to make the arrays 
       'args' : (d_close,), # arguments for the function
       'kwargs' : {'wlen' : 128,
                   'st' : 0.8, 
                   'pd' : 0.8}, # parameters with defaults
       # parameter space for exploration
       'space' : {'wlen' : [32, 64, 128, 256],
                  'st' : [0.5, 0.8, 1.2],
                  'pd' : [0.5, 0.8, 1.0],
                 },
       # how many output arrays it returns and their output range, if any
       'outputs' : [{'range': None, 
                     'type': 'osc',
                     'lag': 0,
                     'abs': 0,
                     'scale': 1,
                     },
                   ], 
       }

regall_single('raw_denoised', d, (d_open, d_high, d_low, d_close))

In [71]:
# the assembly code

data = []
rkeys = sorted(list(register.keys()))

for k in tqdm(rkeys):
    # generate all kwargs combinations for r
    ck= {ka:va for ka,va in register[k]['space'].items()}
    mk= list(it.product(*[v for k,v in sorted(ck.items())]))
    ks = [k for k,v in sorted(ck.items())]
    # start generating
    for kw in mk: 
        kwargs = dict(zip(ks, kw))
        datalines = register[k]['maker'](*register[k]['args'], **kwargs)
        if not isinstance(datalines, tuple):
            datalines = (datalines,)
        # process each data line as specified in the outputs
        pdatalns = []
        for od,dl in zip(register[k]['outputs'], datalines):
            if od['range']: 
                pass # modify dl to fit the [-1 .. 1] range
                dl -= od['range'][0] # -= min
                dl /= (od['range'][1]  - od['range'][0])# /= (max-min)
                dl -= 0.5
                dl *= 2.0 
            if od['lag']:
                # lag dl
                b = dl[1:] - dl[0:-1]
                b = np.hstack([np.array([np.nan]), b])
                dl = b
            if od['scale']:
                dl = scale(dl)
            # todo: other possiblie modifiers, like scaling, etc. 
            pdatalns.append(dl)
        # add the datalines to the big array
        data += pdatalns

# stack the data now 
data = vstack(data)

[-44.571457 -43.400326 -43.148872 -43.311638 -43.12988  -43.261253
 -43.527557 -43.38136  -43.314075 -43.31104  -43.149727 -43.37256
 -43.31867  -43.098522 -43.22916  -43.29213  -44.18127 ] -98.40763
[-41.16341  -40.37345  -40.34564  -40.23014  -40.345196 -40.320034
 -40.804066 -40.234802 -40.09162  -40.584206 -40.38004  -40.10849
 -40.381496 -39.888355 -40.352867 -40.099907 -39.93052  -40.113213
 -40.134216 -40.443363 -40.427456 -40.4723   -40.812855 -40.62317
 -40.47097  -40.736805 -40.95333  -41.14819  -40.989166 -40.4812
 -40.169785 -40.11383  -41.288708] -98.02967
[-38.104332 -36.936253 -37.384632 -37.047024 -37.075367 -37.28129
 -37.158466 -37.360527 -36.646553 -37.706173 -37.93402  -37.707497
 -37.65835  -37.8525   -37.70373  -37.98075  -37.775784 -37.981495
 -37.84817  -37.25334  -36.56648  -36.95833  -37.825245 -37.790913
 -37.187607 -36.75347  -37.664474 -38.02156  -37.028114 -36.786648
 -37.219578 -37.443283 -37.37056  -37.654587 -37.3085   -37.286423
 -37.062424 -37.37463  

[-44.218166 -43.32008  -43.475574 -43.41997  -43.234066 -43.40287
 -43.42576  -43.583504 -43.514496 -43.418236 -43.379093 -43.876774
 -43.76945  -43.38813  -43.17444  -43.239758 -44.26617 ] -98.40763
[-41.434574 -40.226665 -40.272636 -40.31825  -40.32505  -40.640007
 -40.562813 -40.70745  -40.406467 -40.279175 -40.701958 -40.667694
 -40.53066  -40.210316 -40.298737 -40.214466 -40.49867  -40.418835
 -40.591595 -40.705643 -40.725796 -40.545834 -40.345898 -40.77832
 -40.334946 -39.979443 -40.19179  -40.12184  -40.250473 -39.92023
 -39.758194 -39.737713 -40.62496 ] -98.02967
[-37.583733 -36.93098  -37.28625  -37.46709  -37.772312 -37.692425
 -37.914227 -37.9117   -37.65211  -37.4957   -37.76399  -37.077007
 -37.256325 -37.752777 -37.869446 -37.609997 -37.46765  -37.519676
 -37.672867 -37.67057  -37.42116  -37.042866 -37.653564 -37.985676
 -37.36914  -37.203876 -37.1019   -37.039642 -37.270943 -37.548115
 -37.844917 -37.609447 -38.185486 -38.392166 -37.747692 -37.284782
 -36.89724  -36.9252

[-44.471867 -43.68203  -43.587612 -43.327057 -43.23509  -43.370533
 -43.37017  -43.327606 -43.48543  -43.74434  -43.767933 -43.714207
 -43.34824  -43.08662  -43.257683 -43.506676 -44.349586] -98.40763
[-41.602913 -40.461098 -40.024723 -40.28293  -40.616936 -40.394596
 -40.248177 -40.19017  -40.429955 -40.39986  -40.486458 -40.42906
 -40.60921  -39.974815 -40.135452 -40.309826 -40.42496  -40.60831
 -40.54371  -40.36169  -40.093487 -39.89322  -40.036667 -40.659527
 -40.44847  -40.229225 -40.37935  -40.427322 -40.48397  -40.40494
 -40.57113  -40.82259  -41.7799  ] -98.02967
[-38.491848 -37.71013  -37.56279  -37.55137  -37.342854 -37.135162
 -37.1019   -37.302986 -37.47773  -37.273132 -37.10911  -37.25092
 -37.009377 -37.278763 -37.128494 -37.211002 -37.15861  -37.481
 -37.59253  -37.385635 -37.479797 -37.159607 -37.09651  -38.07873
 -37.716877 -37.36329  -36.982323 -37.20318  -37.658543 -37.732014
 -38.047573 -38.243355 -37.66559  -37.837948 -37.53348  -37.349663
 -37.26176  -37.20651  -3

[-44.59452  -43.772144 -43.59568  -43.33715  -43.474518 -43.64614
 -43.7201   -43.76039  -43.59498  -43.540115 -43.69401  -43.52656
 -43.56358  -43.50672  -43.562164 -43.646736 -44.816833] -98.357315
[-41.84999  -40.56156  -40.36766  -40.872055 -41.287907 -41.142937
 -41.0097   -40.984673 -40.67957  -40.73625  -40.720486 -40.79995
 -40.91807  -40.904083 -40.74141  -40.57827  -40.579796 -40.762737
 -40.82132  -40.696415 -40.70259  -40.585518 -40.890755 -40.577908
 -40.679195 -40.733685 -40.880547 -40.592796 -40.41185  -40.539043
 -40.49136  -40.5163   -41.353645] -98.29793
[-38.29203  -37.698807 -37.97225  -37.95614  -38.252712 -38.109444
 -37.75588  -37.173557 -37.40673  -37.754936 -38.343815 -37.828323
 -37.923813 -38.137115 -38.263706 -38.097317 -37.610657 -37.259243
 -37.970165 -37.9883   -37.500443 -37.425842 -37.296535 -37.657017
 -37.4295   -37.345078 -37.39576  -38.29162  -37.821445 -37.918587
 -38.594406 -38.059986 -37.585354 -37.828594 -37.433643 -36.81283
 -37.133327 -37.6218

[-45.046566 -43.595764 -43.469994 -43.53474  -43.542652 -43.600033
 -43.828674 -43.817097 -43.808437 -43.80242  -44.019768 -43.79669
 -43.76875  -43.48973  -43.741207 -44.001114 -44.58843 ] -98.357315
[-41.287407 -40.255672 -40.068188 -40.45565  -40.981808 -40.801056
 -41.050095 -40.748856 -40.240063 -40.814514 -40.8902   -40.9141
 -40.618233 -40.3263   -40.3689   -40.665257 -40.87937  -40.91812
 -40.681828 -40.388863 -40.557114 -40.77738  -40.91165  -40.923435
 -40.42836  -40.484867 -40.895973 -40.649418 -40.594868 -40.60269
 -40.852936 -40.929443 -41.405663] -98.29793
[-38.648216 -37.83999  -37.978138 -37.871017 -37.62797  -37.09686
 -37.176132 -37.380604 -37.5703   -37.47715  -37.59362  -37.766914
 -37.79089  -37.853027 -37.323914 -37.283863 -37.725933 -37.95841
 -37.408966 -37.71768  -37.53753  -37.94066  -37.987957 -37.45021
 -38.11062  -37.92376  -37.62661  -37.970882 -37.87256  -37.747124
 -37.160255 -37.04937  -37.761414 -38.018974 -37.34115  -37.84566
 -37.76048  -36.987976 -3

...

[-45.571144 -44.606155 -44.233383 -44.274063 -44.468075 -44.346695
 -44.312416 -44.39237  -44.454384 -44.234856 -44.258896 -44.212673
 -44.174366 -44.467823 -44.4059   -44.247086 -45.065777] -106.57518
[-42.592594 -41.489418 -41.37703  -41.19633  -41.050365 -41.024597
 -41.238323 -41.03425  -40.804184 -41.339222 -41.296497 -41.029945
 -41.467262 -41.173767 -40.742756 -41.148983 -41.298187 -41.070194
 -41.255493 -41.230595 -40.874657 -41.40706  -41.307606 -41.127098
 -41.70937  -41.998028 -41.45493  -40.854725 -41.185696 -41.07428
 -41.204304 -41.058826 -42.069626] -104.92849
[-39.234386 -38.06773  -38.140747 -38.242065 -38.326714 -38.234512
 -38.187782 -37.99039  -37.636623 -38.04799  -38.161716 -38.545177
 -38.05611  -38.074696 -38.358517 -38.29476  -38.534794 -39.0223
 -38.669243 -37.927734 -37.574738 -37.869667 -38.01153  -38.141502
 -38.72493  -38.558117 -38.314102 -37.830975 -37.736298 -37.79273
 -37.78274  -38.64754  -38.502415 -38.163677 -38.18756  -38.26434
 -37.86481  -37.8145

[-45.746037 -44.5983   -44.169594 -44.01004  -44.149757 -44.39017
 -44.404865 -44.440414 -44.31617  -43.998394 -44.142525 -44.178196
 -44.39927  -44.53679  -44.401024 -44.02443  -45.068954] -106.57518
[-42.271137 -41.206535 -41.357674 -41.18709  -41.466896 -41.369293
 -40.99427  -41.50214  -41.770954 -41.44469  -41.56337  -41.649742
 -41.295906 -41.355583 -41.146805 -40.977474 -40.981434 -41.41179
 -41.302097 -41.281517 -41.001297 -41.199326 -41.57478  -41.41936
 -41.305084 -41.226776 -40.92979  -41.24794  -41.365997 -41.18496
 -40.861736 -41.124786 -41.797382] -104.92849
[-39.543217 -38.502842 -38.813946 -38.308125 -37.806473 -37.689888
 -37.990246 -38.080585 -38.131668 -38.055843 -37.78165  -37.756252
 -38.690693 -38.578182 -38.217087 -38.19565  -38.338955 -38.403687
 -37.98812  -37.882126 -37.9619   -38.639343 -38.662678 -38.244442
 -37.7937   -37.549255 -38.28956  -38.003864 -38.02523  -37.92183
 -37.75271  -38.31744  -38.06331  -38.3037   -38.492764 -38.822525
 -38.171227 -37.7895

[-44.247593 -43.342587 -43.353233 -43.42925  -43.55685  -43.478493
 -43.68562  -43.830368 -43.773087 -43.794415 -43.721203 -43.505463
 -43.19806  -43.442947 -43.385185 -43.42247  -44.01236 ] -106.16701
[-40.770226 -40.394703 -40.77107  -41.033268 -40.90913  -40.60619
 -40.64979  -40.558643 -40.94093  -40.728985 -40.70345  -40.291824
 -40.10021  -40.095642 -40.25636  -40.318604 -40.26775  -40.50872
 -40.541195 -40.503414 -40.45375  -40.815346 -40.751965 -40.53883
 -40.308533 -40.689415 -40.48615  -40.417442 -40.527824 -40.660442
 -40.60726  -40.13698  -41.437134] -103.95833
[-38.16344  -37.490433 -37.725025 -37.644455 -37.329597 -37.392075
 -37.779747 -38.271744 -37.775463 -37.59177  -37.27338  -36.768623
 -37.50922  -37.433838 -37.185375 -37.869053 -37.95862  -37.16789
 -37.71414  -37.0789   -36.638504 -36.82541  -37.29629  -37.826084
 -37.74326  -37.882965 -37.609436 -37.346756 -38.006496 -37.395947
 -37.37849  -37.774883 -37.168015 -36.981606 -37.176228 -37.931522
 -38.41087  -37.823

[-44.806786 -43.574173 -43.395576 -43.588814 -43.424644 -43.47985
 -43.29165  -43.271324 -43.349762 -43.49175  -43.63778  -43.847168
 -43.402718 -43.511627 -43.91775  -43.996326 -44.69651 ] -106.16701
[-42.181007 -40.923656 -40.83464  -40.773003 -40.402927 -40.055153
 -40.37394  -40.6815   -40.697918 -40.633102 -40.383953 -40.470364
 -40.630013 -40.79532  -40.31506  -40.310055 -40.407543 -40.278645
 -40.27002  -40.52128  -40.578423 -40.32813  -40.808876 -40.74108
 -40.59034  -40.88432  -41.043762 -40.543613 -40.577385 -40.655674
 -40.480293 -40.410873 -41.120094] -103.95833
[-39.38417  -37.573452 -37.371754 -37.605648 -37.761143 -37.551125
 -37.334995 -37.509888 -38.529224 -38.479435 -38.420235 -37.68143
 -37.878395 -37.57585  -37.399998 -37.53197  -37.83605  -37.560326
 -38.12575  -37.77372  -37.90164  -37.640636 -37.3211   -37.61388
 -37.86994  -37.52439  -37.69286  -37.79369  -37.554398 -37.49171
 -37.923172 -38.030773 -37.741833 -37.398785 -36.767685 -37.583523
 -37.924458 -37.7087

[-44.156513 -43.450947 -43.665142 -43.548172 -43.52775  -43.522865
 -43.70507  -43.750454 -43.414307 -43.25423  -43.340935 -43.635773
 -43.525425 -43.27984  -43.364532 -43.584106 -44.29178 ] -106.16701
[-41.379364 -40.447712 -40.623764 -40.83654  -40.41816  -40.717003
 -41.16652  -41.22156  -41.20278  -40.64246  -40.37986  -40.64089
 -40.3718   -40.420685 -40.59239  -40.848946 -40.750614 -40.70836
 -40.24691  -40.51808  -41.00751  -40.81037  -40.610153 -40.30844
 -40.261913 -40.77963  -40.71296  -40.81916  -41.004128 -40.5434
 -40.340096 -40.39832  -41.17995 ] -103.95833
[-38.945435 -37.927612 -37.947872 -37.569813 -37.43653  -37.620533
 -37.356686 -37.38118  -37.631226 -38.395542 -37.987465 -38.40895
 -37.840546 -37.701717 -37.62955  -37.28624  -37.52073  -37.930576
 -37.575268 -37.374218 -37.539124 -37.13204  -37.067795 -37.261036
 -37.496147 -37.679462 -37.463688 -37.58089  -37.62828  -37.833416
 -37.44066  -37.30397  -37.741493 -37.153088 -37.69331  -37.46719
 -37.14429  -37.78271 

In [72]:
data.shape

(11291, 51823)

In [73]:
# cut the nans
# find the real start of the data (without NaN)
nc = 0
for cl in range(data.shape[1]):
    a = data[:, cl].reshape(-1)
    if len(a[np.isnan(a)]) > 0:
        nc = cl
# cut off the parts that contain NaN
data = data[:, nc + 1:]
nc = nc+1

In [74]:
# the to_predict part of the data 
tgt = np.vstack([d_open[nc:], 
                 d_high[nc:], 
                 d_low[nc:], 
                 d_close[nc:]])

In [75]:
#imshow(data[:, 0:data.shape[0]]); colorbar();

In [76]:
winlen = 1
sliding_window_jump = 1
predict_time_ahead = 1
sep = 0.000001
binary = 1

# extract the relevant information only through SeleckKBest
def prepare_data(input_source, to_predict, binary=1):

    x = []
    y = []

    ups = []
    downs = []
    mids = []

    for i in range(0, input_source.shape[1]-(winlen+100), sliding_window_jump):
        # form the input
        txs = input_source[:, i:i+winlen]#.astype(np.float64)
        #xs = scale(xs, axis=1)
        xs = txs.ravel()

        # for the output
        now = to_predict[i+winlen-1]# close
        future = to_predict[i+winlen+(predict_time_ahead-1)] # next close

        ys = future-now
        magn = abs(ys)

        if not binary:
            if magn < sep:
                mids.append( (xs, (np.array([1]))) )
            else:
                if ys < 0:
                    downs.append( (xs, (np.array([2]))) )
                else:
                    ups.append( (xs, (np.array([0]))) )
        else:
            if ys <= 0:
                downs.append( (xs, (np.array([1]))) )
            else:
                ups.append( (xs, (np.array([0]))) )
                
        if (i%1000)==0:
            clear_output(wait=True)
            print('({}/{})'.format(i, input_source.shape[1]-winlen))

    if not binary:
        print('Ups: {0} Mids: {1} Downs: {2}'.format(len(ups), len(mids), len(downs)))
        print('Separator: %3.5f' % sep)
    else:
        print('Ups: {0} Downs: {1}'.format(len(ups), len(downs)))
        print('Separator: %3.5f' % sep)        
    print()

    if not binary:
        samplesize = min(len(ups), len(downs), len(mids))
        s1 = rnd.sample(ups, samplesize)
        s2 = rnd.sample(downs, samplesize)
        s3 = rnd.sample(mids, samplesize)
        a = s1 + s2 + s3
    else:
        samplesize = min(len(ups), len(downs))
        s1 = rnd.sample(ups, samplesize)
        s2 = rnd.sample(downs, samplesize)
        a = s1 + s2 
    rnd.shuffle(a)
    x = [x[0] for x in a]
    y = [x[1] for x in a]

    x = np.vstack(x)
    y = np.vstack(y)
    
    x.shape = (x.shape[0], -1)
    cutpoint = int(0.8 * x.shape[0])
    x_train = x[0:cutpoint]
    x_test = x[cutpoint:]
    y_train = y[0:cutpoint]
    y_test = y[cutpoint:]
    
    return x_train, x_test, y_train.reshape(-1), y_test.reshape(-1)

x_train, x_test, y_train, y_test = prepare_data(data, tgt[3])

(51000/51347)
Ups: 25366 Downs: 25881
Separator: 0.00000



In [77]:
[x.shape for x in [x_train, x_test, y_train, y_test]]

[(40585, 11291), (10147, 11291), (40585,), (10147,)]

In [78]:
# Select K Best
print('Extracting best features..')

Extracting best features..


In [79]:
%%time

clf = RidgeClassifier().fit(x_train, y_train)

CPU times: user 8min 46s, sys: 1min 44s, total: 10min 30s
Wall time: 1min 45s


In [80]:
%%time

sfm = SelectFromModel(clf, prefit=True, threshold='mean', max_features=320)

CPU times: user 11 µs, sys: 2 µs, total: 13 µs
Wall time: 16.5 µs


In [81]:
%%time

n_features = sfm.transform(x_train).shape[1]
n_features

CPU times: user 603 ms, sys: 315 µs, total: 604 ms
Wall time: 604 ms


320

In [82]:
%%time

data = sfm.transform(data.T).T

CPU times: user 411 ms, sys: 8.59 ms, total: 419 ms
Wall time: 418 ms


In [83]:
print('done.')

done.


In [84]:
#data = scale(data, axis=1)

In [85]:
#imshow(data[:, 0:data.shape[0]*2]); colorbar();

In [86]:
np.save("data/data_%s.npy" % dataname, data)
np.save("data/data_%s_targets.npy" % dataname, tgt)

In [87]:
#np.min(data, axis=1)

In [88]:
#np.max(data, axis=1)